<a href="https://colab.research.google.com/github/Computer-CGuy/MusgraveTextureNeuralRenderer/blob/master/SKIN_CANCER_%7C_NeuralRenderer_Parameters_to_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch  
import matplotlib.pyplot as plt  
import numpy as np  
import torch.nn.functional as func
import torch.nn.functional as F
import PIL.ImageOps  
from torch import nn  
from torchvision import datasets,transforms   
import requests  
from PIL import Image  
import torchvision.models as models
import os
import json
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
from google.colab.patches import cv2_imshow
import torchvision

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4,1000)
        self.fc12 = nn.Linear(1000,6000)
        self.fc2 = nn.Linear(6000,9000)
        # self.fc5 = nn.Linear(310,800)
        # self.fc11 = nn.Linear(810,1100)
        # self.fc6 = nn.Linear(1110,1500)
        # self.fc7 = nn.Linear(1500,2000)
        # self.fc13 = nn.Linear(2000,4000)
        self.fc14 = nn.Linear(10000,1024*3)
        self.fc8 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.fc9 = nn.Upsample(scale_factor=2, mode='nearest')
        self.fc10 = nn.Upsample(scale_factor=2, mode='bilinear')
        # self.lstm = nn.LSTM(3,10,4)
        self.conv1 = torch.nn.Conv2d(3, 40, 5)
        self.conv2 = torch.nn.Conv2d(40, 80,1) 
        self.conv3 = torch.nn.ConvTranspose2d(80,20,1)
        self.conv4 = torch.nn.ConvTranspose2d(20,3,5)

        self.dropout = 0
    def forward(self, x, sig = False):
        x1 = x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc12(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        x = torch.cat((x1, x), dim=1)   
        x = (nn.functional.leaky_relu(self.fc14(x)))
        # print(x.shape)
        x = torch.reshape(x,(-1,3,32,32))
        x = nn.functional.leaky_relu(self.fc8(x)) #56
        # print(x.shape)
        

        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.fc9(x)) #112
        # print(x.shape)
        x = nn.functional.relu(self.conv2(x))
            
        x = nn.functional.relu(self.fc10(x))  #224
        # print(x.shape)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.relu(self.conv4(x))
        if(sig==True):
          x = torch.sigmoid(x)
        #x = self.fc5(x)
        return x

render  = Net().cuda()

In [ ]:
from PIL import Image

In [ ]:
class MusgraveDataset(Dataset):
    def __init__(self,transforms):
        global l
        self.X = l
        self.t = transforms
    def __getitem__(self, index):
        # print(index)
        img = cv2.imread("./drive/My Drive/MusgraveTextureNeuralRenderer/Generator2/pic%0.2d.jpg"%index)
        # img = cv2.resize(img,(256,244))
        # img = img.mean(2)
        # img = np.expand_dims(img,0)
        img = np.transpose(img,(2,0,1))
        return (torch.Tensor(self.X[index])), (torch.Tensor(img))
    
    def __len__(self):
        return len(self.X)

In [ ]:
!ls "./drive/My Drive/MusgraveTextureNeuralRenderer/Generator/*.pkl"

In [ ]:
import pickle
with open("./drive/My Drive/MusgraveTextureNeuralRenderer/out.pkl","rb") as f:
    l = (pickle.load(f))

In [ ]:
transform = transforms.Compose([torchvision.transforms.Resize((224,224))])
train = MusgraveDataset(transforms = transform)
train_loader =DataLoader(dataset=train, batch_size=10, shuffle=True, num_workers=2)

In [ ]:
import torch.optim as optim

# criterion = nn.MSELoss(reduction='sum')
criterion = nn.L1Loss()
optimizer = optim.Adam(render.parameters(), lr=0.0001)

# optimizer = optim.SGD(render.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def show(img):
    img = img.detach().cpu().numpy()[0]
    img  = np.transpose(img,(1,2,0))
    print(img.shape)
    cv2_imshow(img)

In [ ]:
render = models.alexnet(pretrained=True).cuda()

In [ ]:
render = models.resnet18(pretrained=True).cuda()

In [ ]:
render = models.densenet161(pretrained=True).cuda()

In [ ]:
render

In [ ]:
render.fc = nn.Sequential(nn.Linear(512,4).cuda(),
                          nn.ReLU()) 

In [ ]:
render.classifier[6] = nn.Sequential(nn.Linear(4096,4).cuda(),
                          nn.ReLU()).cuda()

In [ ]:
render.classifier = nn.Linear(2208,4).cuda()

In [ ]:
render

In [ ]:
for i,data in enumerate(train_loader):
    inputs, labels = data[0].cuda(),data[1].cuda()
    inputs[:,0]/=12
    inputs[:,1]/=16
    inputs[:,2]/=4
    inputs[:,3]/=3
    labels/=255
    optimizer.zero_grad()
    # outputs = render(inputs,False)
    output = render(labels)
    loss = criterion(output, inputs)
    loss.backward()
    print(loss.item())
    
    if(i%10==0):
        print(output,inputs)
    #     # print(outputs.mean())
    #     # output = norm(outputs)
    #     # show(outputs*255)
    #     # show(labels*255)
    optimizer.step()
    

0.5246191024780273
tensor([[-0.6661,  0.1047,  0.4848,  0.1961],
        [-0.2359, -0.0023,  0.2348,  0.1686],
        [-0.2359, -0.0023,  0.2348,  0.1686],
        [-0.1777,  0.3438,  0.0033,  0.1477],
        [-0.2359, -0.0023,  0.2348,  0.1686],
        [-0.4070,  0.1114,  0.3661,  0.6669],
        [-0.2359, -0.0023,  0.2348,  0.1686],
        [-0.0477,  0.1311,  0.6115,  0.4744],
        [-0.5495, -0.0266,  0.1919,  0.9279],
        [-0.2359, -0.0023,  0.2348,  0.1686]], device='cuda:0',
       grad_fn=<AddmmBackward>) tensor([[0.8333, 0.4375, 0.2500, 1.0000],
        [0.5833, 1.0000, 0.0000, 0.3333],
        [0.5833, 0.2500, 0.0000, 0.6667],
        [0.0000, 0.9375, 0.2500, 1.0000],
        [0.8333, 0.7500, 0.0000, 1.0000],
        [0.0000, 0.0625, 1.0000, 1.0000],
        [0.1667, 0.8750, 0.0000, 0.0000],
        [0.2500, 0.0625, 0.2500, 1.0000],
        [0.6667, 0.3750, 1.0000, 0.3333],
        [0.3333, 0.3125, 0.0000, 1.0000]], device='cuda:0')
0.5327011942863464
0.502264797687

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/RenderResNet.pth")

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/AlexNet.pth")

In [ ]:
def norm(out):
    out-=out.mean()
    out/=out.max()
    return out

In [ ]:
inputs.shape,output.shape

(torch.Size([4, 4]), torch.Size([4, 1000]))

In [ ]:
NetEpochs = 10000
start, stop, step = 7,10,1
Dataset = [x for x in range(1000)]
# Dataset = [0,1]
# Dataset = [0]
done = False
for data in Dataset:
  
  tdata = torch.Tensor([p]).cuda()
  X = model(tdata)
  # print(X)t
  X_CNN = cnn(X)
  # X = X*(255/X.mean())

  X_CNN2 = cnn2(X)
  
  img = process_image(X)
  # print(data,end=" ")
  # print("[ AlexNet "+str(dat[int(torch.argmax(X_CNN).detach().cpu().numpy())]),end="] ")
  # print("[ ResNet "+str(dat[int(torch.argmax(X_CNN2).detach().cpu().numpy())]),end="]")
  
  # cv2_imshow(img)
  cv2.imwrite("dog.jpg",img)
  os.system("python gradcam.py --image-path=dog.jpg --use-cuda")
  cp(data)
  updateJson(data,X_CNN,X_CNN2)
  if((data%10)==0):
    print(data)
  # break
  # imgR = cv2.imread("cam.jpg")
  # cv2_imshow(imgR)
